In [1]:
import sys
sys.path.append("src")

import torch
import lightning as L
torch.set_float32_matmul_precision('medium')

In [2]:
from char_rnn.data import TextDataset, TextDataModule
from char_rnn.model import CharRNN, CharRNNModule

# Data

In [3]:
txt_path = "data/anna.txt"

with open(txt_path, "r") as f:
    text = f.read()

char = tuple(set(text))

In [4]:
seq_length = 50
batch_size = 64

In [5]:
dataset = TextDataset(txt_path, seq_length)

In [6]:
dataset[0]

(tensor([10., 26., 66., 34., 52., 35., 46., 72., 28., 65., 65., 65., 43., 66.,
         34., 34.,  0., 72., 81., 66.,  4., 58., 40., 58., 35., 21., 72., 66.,
         46., 35., 72., 66., 40., 40., 72., 66., 40., 58., 11., 35.,  5., 72.,
         35., 30., 35., 46.,  0., 72., 68., 16.]),
 tensor([26., 66., 34., 52., 35., 46., 72., 28., 65., 65., 65., 43., 66., 34.,
         34.,  0., 72., 81., 66.,  4., 58., 40., 58., 35., 21., 72., 66., 46.,
         35., 72., 66., 40., 40., 72., 66., 40., 58., 11., 35.,  5., 72., 35.,
         30., 35., 46.,  0., 72., 68., 16., 26.]))

In [7]:
datamodule = TextDataModule(txt_path, seq_length, batch_size=batch_size)
datamodule.setup()

In [8]:
train_data = datamodule.train_dataloader()

inp_embed, target_embed = next(iter(train_data))

In [9]:
inp_embed.shape, target_embed.shape

(torch.Size([64, 50, 83]), torch.Size([64, 50, 83]))

# Model

In [ ]:
model = CharRNN(tokens=char)

hidden = model.init_hidden(batch_size)
hidden = tuple([each for each in hidden])

In [ ]:
model(inp_embed, hidden)

(tensor([[ 0.0129,  0.0692,  0.0457,  ...,  0.0225, -0.0200,  0.0427],
         [ 0.0105,  0.0891,  0.0359,  ...,  0.0330, -0.0259,  0.0439],
         [-0.0175,  0.0662,  0.0234,  ...,  0.0409, -0.0284,  0.0245],
         ...,
         [-0.0006,  0.0552,  0.0708,  ..., -0.0005, -0.0623,  0.0473],
         [ 0.0268,  0.0969,  0.0400,  ...,  0.0435, -0.0202,  0.0565],
         [ 0.0090,  0.0843,  0.0400,  ...,  0.0474, -0.0353,  0.0606]],
        grad_fn=<AddmmBackward0>),
 (tensor([[[ 0.0280,  0.0027,  0.0479,  ..., -0.0207, -0.0519,  0.0083],
           [ 0.0218, -0.0026,  0.0503,  ..., -0.0055, -0.0595, -0.0089],
           [ 0.0406, -0.0056,  0.0542,  ..., -0.0056, -0.0275,  0.0186],
           ...,
           [ 0.0213, -0.0071,  0.0234,  ...,  0.0042, -0.0362,  0.0102],
           [ 0.0308,  0.0065,  0.0493,  ..., -0.0097, -0.0464, -0.0077],
           [ 0.0192, -0.0047,  0.0353,  ..., -0.0070, -0.0533, -0.0082]],
  
          [[ 0.0103,  0.0012, -0.0206,  ..., -0.0374, -0.0353, -0.

# Training

In [ ]:
module = CharRNNModule(model)

TypeError: CharRNNModule.__init__() missing 2 required positional arguments: 'loss_fn' and 'optimizer'

In [ ]:
trainer = L.Trainer(
    accelerator="cuda", 
    max_epochs=10, 
    gradient_clip_val=5.0,
    logger=False
)

trainer.fit(module, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | CharRNN          | 896 K  | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
896 K     Trainable params
0         Non-trainable params
896 K     Total params
3.587     Total estimated model params size (MB)
5         Modules in train mode
0         Modules in eval mode


Epoch 1:   1%|          | 311/27917 [00:03<04:57, 92.90it/s, train_loss=1.370, val_loss=1.190]   


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
3200*83

265600